In [1]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic # Not used in this specific example, but imported
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda # Core LCEL components
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal # For defining literal types in Pydantic

# Load environment variables from a .env file. 🌍
# This ensures your OpenAI API key is loaded securely.
load_dotenv()

True

In [2]:
# Initialize the ChatOpenAI language model. 🤖
# This model will be used for both sentiment classification and response generation.
model = ChatOpenAI()

# Initialize a StrOutputParser. 📄
# This parser is used to extract the raw string content from the final LLM response
# in the conditional branches.
parser = StrOutputParser()

In [3]:
# Define the Pydantic schema for feedback sentiment. 📝
# This `Feedback` class specifies that the sentiment must be either 'positive' or 'negative'.
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

In [4]:
# Create a PydanticOutputParser for the Feedback schema. ⚙️
# This parser will generate instructions for the LLM to output sentiment as a structured object,
# and then validate and parse the LLM's output into a `Feedback` object.
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [5]:
# Define the prompt template for sentiment classification. ✍️
# It takes 'feedback' text and includes `format_instruction` from `parser2`
# to guide the LLM to output the sentiment in the specified Pydantic format.
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into positive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

In [6]:
# Create the classifier chain. 🔗
# This chain takes feedback text, classifies its sentiment using the model,
# and parses the output into a `Feedback` Pydantic object.
classifier_chain = prompt1 | model | parser2

In [7]:
# Define the prompt template for a positive response. 📝
# This prompt takes the original 'feedback' text to craft a suitable positive reply.
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

# Define the prompt template for a negative response. 📝
# This prompt takes the original 'feedback' text to craft a suitable negative reply.
prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [8]:
# Define the CONDITIONAL BRANCHING logic using RunnableBranch. 🚦
# `RunnableBranch` takes a series of (condition, runnable) pairs and an optional default runnable.
# - The first argument of each tuple is a `lambda` function that acts as the condition.
#   It receives the output of the *previous* runnable in the overall chain (which is the `Feedback` object from `classifier_chain`).
# - The second argument is the runnable to execute if the condition is true.
#   Here, if `x.sentiment` is 'positive', `prompt2 | model | parser` is executed.
#   If `x.sentiment` is 'negative', `prompt3 | model | parser` is executed.
# - `RunnableLambda(lambda x: "could not find sentiment")` acts as a fallback if neither condition is met.
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment") # Default fallback if sentiment is not positive/negative
)

In [9]:
# Combine the classifier chain and the branching chain into a single, overall chain. 🔗
# The output of `classifier_chain` (the `Feedback` object) is passed directly to `branch_chain`.
chain = classifier_chain | branch_chain

In [10]:
# Invoke the entire chain with sample feedback. 🚀
# The chain will classify the sentiment and then generate an appropriate response based on it.
print(chain.invoke({'feedback': 'This is a worst iphone'}))

# Print an ASCII representation of the chain's graph. 📈
# This visualization clearly shows the initial classification step,
# followed by the branching logic that leads to different response generation paths.
# It's an excellent tool for understanding the flow of conditional workflows.
chain.get_graph().print_ascii()

"I'm sorry to hear that you had a negative experience. We strive to provide top-notch service and I would love to hear more about what specifically went wrong so we can address it and improve in the future. Please feel free to reach out to me directly so we can discuss this further. Thank you for bringing this to our attention."
    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOpenAI |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |  